# Weak Mixed k-Metric Dimensions
Za začetek napišimo CLP program, ki bo vrnil $wmdim_k(G)$ in pripadajoč "resolving set" $S$. Pred tem pa definirajmo še razdaljo med vozlišči in povezavami.

In [1]:
from sage.numerical.mip import MixedIntegerLinearProgram

def razdalja(moznost, vozlisce, G):
    c, a = moznost
    if c == 'e':
        U, V = a
        return min(G.distance(U, vozlisce), G.distance(V, vozlisce))
    else:
        return G.distance(a, vozlisce)

In [2]:
def CLP_weak_k_dim(G, k):
    p = MixedIntegerLinearProgram(maximization=False)
    x = p.new_variable(binary=True)

    V = G.vertices()
    E = G.edges(labels=False)

    moznosti = [('v', v) for v in V] + [('e', e) for e in E]

    p.set_objective(sum(x[v] for v in V))
    
    for a, b in Combinations(moznosti, 2):
        p.add_constraint(
            sum(abs(razdalja(a, v, G) - razdalja(b, v, G)) * x[v] for v in V) >= k
            )

    wmdim_k = p.solve()
    mnozica_S = [v for v in V if p.get_values(x[v]) > 0.5]

    return (wmdim_k, mnozica_S)

Sedaj definirajmo še funkcijo $\kappa''(G)$, ki je po definiciji največja vrednost $k$ za katero je $wmdim_k(G)$ še definirana.

In [3]:
def kappa_2_crti(G):
    k = 1
    while True:
        try:
            CLP_weak_k_dim(G, k)
            k += 1
        except:
            return k - 1

## Cikli

In [4]:
def cikli_do_n(n):
    for i in range(3, n + 1):
        G = graphs.CycleGraph(i) 
        kappa_2crti = kappa_2_crti(G)
        print(f"- velikost: {i} : kappa = {kappa_2crti}")
        for k in range(1, kappa_2crti + 1):
            wmdim, _ = CLP_weak_k_dim(G, k)
            print(f"  k = {k}, wmdim = {wmdim}")

In [5]:
cikli_do_n(10)

- velikost: 3 : kappa = 1
  k = 1, wmdim = 3.0
- velikost: 4 : kappa = 2
  k = 1, wmdim = 3.0
  k = 2, wmdim = 4.0
- velikost: 5 : kappa = 2
  k = 1, wmdim = 3.0
  k = 2, wmdim = 5.0
- velikost: 6 : kappa = 3
  k = 1, wmdim = 3.0
  k = 2, wmdim = 4.0
  k = 3, wmdim = 6.0
- velikost: 7 : kappa = 3
  k = 1, wmdim = 3.0
  k = 2, wmdim = 5.0
  k = 3, wmdim = 7.0
- velikost: 8 : kappa = 4
  k = 1, wmdim = 3.0
  k = 2, wmdim = 4.0
  k = 3, wmdim = 6.0
  k = 4, wmdim = 8.0
- velikost: 9 : kappa = 4
  k = 1, wmdim = 3.0
  k = 2, wmdim = 5.0
  k = 3, wmdim = 7.0
  k = 4, wmdim = 9.0
- velikost: 10 : kappa = 5
  k = 1, wmdim = 3.0
  k = 2, wmdim = 4.0
  k = 3, wmdim = 6.0
  k = 4, wmdim = 8.0
  k = 5, wmdim = 10.0


Glede na dani vzorec sklepam, da je $$\kappa''(G)= \Big\lfloor \frac{n}{2} \Big\rfloor.$$ 

Za $wmdim_k(G)$ pa lahko opazimo naslednjo formulo: $$wmdim_1(G) = 3,$$
$$wmdim_k(G) = \begin{cases}
2k, &\text{če} \ n \ \text{sod}\\
2k + 1, &\text{če} \ n \ \text{lih}
\end{cases}$$

## Hiperkocke

In [6]:
def hiperkocke_do_n(n):
    for i in range(1, n + 1):
        G = graphs.CubeGraph(i)
        kappa_2crti = kappa_2_crti(G)
        print(f"- velikost: {i} : kappa = {kappa_2crti}")
        for k in range(kappa_2crti + 1):
            wmdim, _ = CLP_weak_k_dim(G, k)
            print(f"  k = {k}, wmdim = {wmdim}")

In [7]:
hiperkocke_do_n(4)

- velikost: 1 : kappa = 1
  k = 0, wmdim = 0.0
  k = 1, wmdim = 2.0
- velikost: 2 : kappa = 2
  k = 0, wmdim = 0.0
  k = 1, wmdim = 3.0
  k = 2, wmdim = 4.0
- velikost: 3 : kappa = 4
  k = 0, wmdim = 0.0
  k = 1, wmdim = 3.0
  k = 2, wmdim = 4.0
  k = 3, wmdim = 7.0
  k = 4, wmdim = 8.0
- velikost: 4 : kappa = 8
  k = 0, wmdim = 0.0
  k = 1, wmdim = 4.0
  k = 2, wmdim = 5.0
  k = 3, wmdim = 7.0
  k = 4, wmdim = 8.0
  k = 5, wmdim = 11.0
  k = 6, wmdim = 14.0
  k = 7, wmdim = 15.0
  k = 8, wmdim = 16.0


Na podlagi rezultatov za prvih nekaj $n$ lahko sklepamo, da je $\kappa''(G) = 2^{n-1}$. Za $wmdim_k(G)$ zaenkrat težko sklepam.